In [136]:
import pandas as pd

train_df = pd.read_csv("./data/train_2.csv", na_values=['NA','?', '-1'])
test_df = pd.read_csv("data/test_2.csv", na_values=['NA','?', '-1'])


In [137]:
train_df.shape
test_df.shape


(399335, 201)

In [138]:
little_y = train_df['target']

In [141]:
train_df.dtypes

id          int64
target      int64
A_1       float64
A_2       float64
B_1       float64
B_2       float64
B_3       float64
B_4       float64
B_5        object
B_6        object
B_7       float64
B_8       float64
B_9       float64
B_10      float64
B_11      float64
B_12        int64
B_13      float64
B_14       object
B_15       object
C_1       float64
C_2       float64
C_3       float64
C_4       float64
C_5       float64
C_6       float64
C_7       float64
C_8       float64
C_9       float64
C_10      float64
C_11      float64
           ...   
D_137     float64
D_138     float64
D_139     float64
D_140     float64
D_141     float64
D_142     float64
D_143     float64
D_144     float64
D_145     float64
D_146     float64
D_147     float64
D_148     float64
D_149     float64
D_150     float64
D_151     float64
D_152     float64
D_153     float64
D_154     float64
D_155     float64
D_156     float64
D_157     float64
D_158     float64
D_159     float64
D_160     float64
D_161     

In [77]:
print(train_df.B_5.unique())
print(train_df.B_6.unique())
print(train_df.B_14.unique())
print(train_df.B_15.unique())


[nan 'B' 'C' 'D' 'A' 'F' 'E']
[nan 'B' 'A' 'C']
['A' nan 'B']
[nan 'Q' 'b' 'j' 'D' 'C' 'c' 'f' 'T' 'N' 'F' 'Y' 'J' 'g' 'A' 'U' 'S' 'l'
 'm' 'n' 'K' 'O' 'I' 'X' 'B' 'H' 'a' 'p' 'R' 'L' 'V' 'd' 'E' 'P' 'G' 'e'
 'W' 'M' 'i' 'o' 'h' 'k']


In [ ]:
from sklearn import preprocessing
minmax = pd.DataFrame()
standard = pd.DataFrame()
cont = list()
cont = ["B_3", "B_4", "B_8", "B_10", "B_11", "B_12", "C_1", "C_2", "C_3", "C_4", "C_5", "C_6", "C_7", "C_8", "C_9", "C_10", "C_11", "C_12", "C_13", "C_14", "C_15", "C_16", "C_17"] 
for i in range(166):
    col = "D_" + str(i+1)
    cont.append(col)
for category in cont:
    for row in train_df[train_df[category].isnull()].itertuples():
      train_df.loc[row[0],category] = train_df[category].mean()
    minmax[category+"_minmax"] = preprocessing.minmax_scale(train_df[category])
    standard[category+"_standard"] = preprocessing.scale(train_df[category])

In [130]:
import category_encoders as ce
encoder = ce.TargetEncoder(cols=["B_5", "B_6", "B_14", "B_15"], return_df=True)
X = train_df
y = train_df['target']
train2_df = encoder.fit_transform(X,y)
train3_df = pd.get_dummies(train_df, dummy_na=True, columns=["B_5", "B_6", "B_14", "B_15"])
train_df = train_df.drop(["id", "B_5", "B_6", "B_14", "B_15"], axis=1)
train2_df = train2_df.drop(["id", "target", "A_1", "A_2", "B_1", "B_2", "B_3", "B_4", "B_7", "B_8", "B_9", "B_10", "B_11", "B_12", "B_13", "C_1", "C_2", "C_3", "C_4", "C_5", "C_6", "C_7", "C_8", "C_9", "C_10", "C_11", "C_12", "C_13", "C_14", "C_15", "C_16", "C_17"], axis=1)
train3_df = train3_df.drop(["id", "target", "A_1", "A_2", "B_1", "B_2", "B_3", "B_4", "B_7", "B_8", "B_9", "B_10", "B_11", "B_12", "B_13", "C_1", "C_2", "C_3", "C_4", "C_5", "C_6", "C_7", "C_8", "C_9", "C_10", "C_11", "C_12", "C_13", "C_14", "C_15", "C_16", "C_17"], axis=1)
train_df = pd.concat([train_df, train2_df], axis=1)
train_df = pd.concat([train_df, train3_df], axis=1)

In [131]:
train_df = pd.concat([train_df, minmax], axis=1)
train_df = pd.concat([train_df, standard], axis=1)

In [132]:
train_df.shape

(400665, 137)

In [134]:

# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
# load data
dataset = train_df

# split data into X and y
X = dataset.drop(['target'], axis=1)
Y = dataset['target']
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=194, seed=None,
       silent=False, subsample=1)
model.fit(X_train, y_train)

ValueError: feature_names must be unique

In [16]:
%%bash
pip3 install category_encoders

  Using cached https://files.pythonhosted.org/packages/f7/d3/82a4b85a87ece114f6d0139d643580c726efa45fa4db3b81aed38c0156c5/category_encoders-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/0c/5f61f1a3d4385d6bf83b83ea495068857ff8dfb89e74824c6e9eb63286d8/patsy-0.5.1-py2.py3-none-any.whl
